In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('raw_data.csv', skipinitialspace=True, sep=';')
df.head()

,Год,Время года,Климат,Город,Страна,Способ охлаждения,Режим при смешанном типе охлаждения,Способ обогрева,Возраст,Пол,...,Скорость воздуха,Рост,Вес,Занавески,Вентилятор,Окно,Двери,Отопление,Среднемесячная температура на улице,Количество рекламаций
0,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,NaN,NaN,22.0,NaN,...,0.26,NaN,NaN,1.0,1.0,NaN,NaN,NaN,28.9,0
1,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,NaN,NaN,40.0,NaN,...,0.12,NaN,NaN,0.0,0.0,NaN,NaN,NaN,84.0,0
2,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,NaN,NaN,42.0,NaN,...,0.08,NaN,NaN,0.0,0.0,NaN,NaN,NaN,32.8,0
3,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,NaN,NaN,27.0,NaN,...,27.16,NaN,NaN,1.0,1.0,NaN,NaN,NaN,28.9,1
4,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,NaN,NaN,46.0,NaN,...,63.83,NaN,NaN,1.0,1.0,NaN,NaN,NaN,32.8,0


In [4]:
df.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)

In [5]:
df['режим_при_смешанном_типе_охлаждения'] = df['режим_при_смешанном_типе_охлаждения'].fillna('нет')
df['способ_обогрева'] = df['способ_обогрева'].fillna('нет')

In [6]:
df.to_csv('raw_data.csv')

In [7]:
# заполняем оценку комфорта
gbl = ['климат', 'время_года', 'способ_охлаждения', 'способ_обогрева', 'режим_при_смешанном_типе_охлаждения']
gb = df.groupby(gbl)['оценка_комфорта']
md = gb.median()
for key in md.index:
    condition = df['оценка_комфорта'] != -1
    for i in range(len(gbl)):
        condition = condition & (df[gbl[i]] == key[i])
    if not md.isna()[key]:
        df['оценка_комфорта'][condition] = df['оценка_комфорта'][condition].fillna(md[key])
    else:
        df['оценка_комфорта'][condition] = df['оценка_комфорта'][condition].fillna(df['оценка_комфорта'].median())

In [8]:
# заполняем возраст
gb = df.groupby('город')['возраст']
md = gb.median()
for city in md.index:
    if not md.isna()[city]:
        df['возраст'][df['город'] == city] = df['возраст'][df['город'] == city].fillna(md[city])
    else:
        df['возраст'][df['город'] == city] = df['возраст'][df['город'] == city].fillna(df['возраст'].median())

In [9]:
# заполняем пол
gb = df.groupby('город')['пол']
glbrel = df['пол'].value_counts()['Мужской'] / df['пол'].value_counts()['Женский']
for key, it in gb:
    vc = it.value_counts()
    if vc.size and vc['Мужской'] and vc['Женский']:
        rel = vc['Мужской'] / vc['Женский']
    else:
        rel = glbrel
    ln = len(it)
    f = int(round(ln / (rel + 1)))
    m = ln - f
    df['пол'].fillna({it.index[i]: ('Мужской' if i < m else 'Женский') for i in range(ln)}, inplace=True)

In [10]:
# заполняем рост
df["возраст"] = df['возраст'].astype(int)
df.loc[df['рост'] <= df['рост'].quantile(0.01), 'рост'] = np.nan
gb1 = df.groupby(['возраст'])['рост']
for key, item in gb1:
    key_mean = gb1.get_group(key).median()
    df["рост"] = df["рост"].fillna(key_mean)

In [11]:
# заполняем вес
gb2 = df.groupby(['возраст'])['вес']
for key, item in gb2:
    key_mean = gb2.get_group(key).median()
    df["вес"] = df["вес"].fillna(key_mean)

In [12]:
# заполняем ощущение движения воздуха
gb = df.groupby(['способ_обогрева', 'способ_охлаждения', 'режим_при_смешанном_типе_охлаждения', 'климат', 'время_года'])['ощущение_движения_воздуха_(bool)']
glbrel = df['ощущение_движения_воздуха_(bool)'].value_counts()[0] / df['ощущение_движения_воздуха_(bool)'].value_counts()[1]
for key, it in gb:
    vc = it.value_counts()
    if vc.size and vc[0] and vc[1]:
        rel = vc[0] / vc[1]
    else:
        rel = glbrel
    ln = len(it)
    o = int(round(ln / (rel + 1)))
    z = ln - o
    df['ощущение_движения_воздуха_(bool)'].fillna({it.index[i]: (1 if i < o else 0) for i in range(ln)}, inplace=True)

In [13]:
# заполняем температуру воздуха на улице
gbl = ['климат', 'время_года']
gb = df.groupby(gbl)['температура_воздуха_на_улице']
md = gb.median()
for key in md.index:
    condition = df['температура_воздуха_на_улице'] != -1
    for i in range(len(gbl)):
        condition = condition & (df[gbl[i]] == key[i])
    if not md.isna()[key]:
        df['температура_воздуха_на_улице'][condition] = df['температура_воздуха_на_улице'][condition].fillna(md[key])
    else:
        df['температура_воздуха_на_улице'][condition] = df['температура_воздуха_на_улице'][condition].fillna(df['температура_воздуха_на_улице'].median())

In [17]:
# заполняем занавески
vc = df['занавески'].value_counts()
rel = vc[1] / vc[0]
z = int(round(len(df) / (rel + 1)))
o = ln - z
df['занавески'].fillna({i: (1 if i < o else 0) for i in df.index}, inplace=True)

In [18]:
df.to_csv('corrected_data.csv')

In [19]:
(~df.isna()).sum()

год                                            484
время_года                                     484
климат                                         484
город                                          484
страна                                         484
способ_охлаждения                              484
режим_при_смешанном_типе_охлаждения            484
способ_обогрева                                484
возраст                                        484
пол                                            484
ощущение_температуры                           484
ощущение_температуры_(bool)                    484
предпочтительное_изменение_температуры         484
ощущение_движения_воздуха_(bool)               484
предпочтительное_изменение_движения_воздуха    484
оценка_комфорта                                484
утепление                                      484
температура_воздуха_в_помещении                484
температура_воздуха_на_улице                   484
rh                             